In [1]:
# Library imports
import tensorflow as tf
import numpy as np
keras = tf.keras
import matplotlib.pyplot as plt
import os
import sys

In [2]:
# Add src directory to path for local imports
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Local imports
from utils import gpu_grow_memory

In [3]:
gpu_grow_memory()

In [4]:
@tf.function
def simple_nn_layer(x, y):
    return tf.nn.relu(tf.matmul(x, y))

In [5]:
x = tf.random.uniform((3,3))
y = tf.random.uniform((3,3))

simple_nn_layer(x, y)

<tf.Tensor: id=23, shape=(3, 3), dtype=float32, numpy=
array([[0.5328578 , 0.41398597, 0.14595816],
       [1.6773702 , 0.71510756, 0.5577528 ],
       [1.110822  , 0.627371  , 0.5181678 ]], dtype=float32)>

In [6]:
simple_nn_layer

In [7]:
def linear_layer(x):
    return 2*x + 1

@tf.function
def deep_net(x):
    return tf.nn.relu(linear_layer(x))

deep_net(tf.constant((1,2,3)))

<tf.Tensor: id=36, shape=(3,), dtype=int32, numpy=array([3, 5, 7])>

In [8]:
import timeit

conv_layer = tf.keras.layers.Conv2D(100, 3)

@tf.function
def conv_fn(image):
    return conv_layer(image)

image = tf.zeros([1, 200, 200, 100])

conv_layer(image)
conv_fn(image)

print(f'Eager conv:   ', timeit.timeit(lambda: conv_layer(image), number=10))
print(f'Function conv:', timeit.timeit(lambda: conv_fn(image), number=10))

Eager conv:    0.0030880000000195196
Function conv: 0.0025162000000023


In [10]:
lstm_cell = tf.keras.layers.LSTMCell(10)
@tf.function
def lstm_fn(input, state):
    return lstm_cell(input, state)

input = tf.zeros([10, 10])
state = [tf.zeros([10, 10])] * 2

lstm_cell(input, state)
lstm_fn(input, state)

print('Eager LSTM:    ', timeit.timeit(lambda: lstm_cell(input, state), number=10))
print('Function LSTM: ', timeit.timeit(lambda: lstm_fn(input, state), number=10))

Eager LSTM:     0.007383000000004358
Function LSTM:  0.003946200000029876


In [12]:
@tf.function
def square_if_positive(x):
    if x > 0:
        return x * x
    else:
        return 0
    
print(f'square_if_positive(2) = {square_if_positive(tf.constant(2))}')
print(f'square_if_positive(-2) = {square_if_positive(tf.constant(-2))}')

square_if_positive(2) = 4
square_if_positive(-2) = 0


In [18]:
@tf.function
def sum_even(items):
    s = 0
    for c in items:
        if (c % 2) > 0:
            continue
        else:
            s += c
    return s

sum_even(tf.constant([1,2,3,4,5,6]))

<tf.Tensor: id=852, shape=(), dtype=int32, numpy=12>

In [16]:
tf.constant([1,2,3,4,5,6])

<tf.Tensor: id=720, shape=(6,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6])>

In [21]:
@tf.function
def fizzbuzz(n):
    for i in tf.range(1, n+1):
        if tf.equal(i % 15, 0):
            tf.print('FizzBuzz')
        elif tf.equal(i % 3, 0):
            tf.print('Fizz')
        elif tf.equal(i % 5, 0):
            tf.print('Buzz')
        else:
            tf.print(i)

fizzbuzz(tf.constant(15))

1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14
FizzBuzz


In [22]:
fizzbuzz(15)

1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14
FizzBuzz


In [65]:
def prepare_mnist_data(x, y):
    x = tf.cast(x, tf.float32) / 255.0
    y = tf.cast(y, tf.int32)
    return x, y

def mnist_dataset():
    (x, y), _ = tf.keras.datasets.mnist.load_data()
    ds = tf.data.Dataset.from_tensor_slices((x,y))
    ds = ds.map(prepare_mnist_data)
    ds = ds.take(20000).shuffle(20000).batch(100)
    return ds

In [66]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28,)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense((10)),
])
model.build()
optimizer = tf.keras.optimizers.Adam()

In [67]:
calc_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
calc_acc = tf.keras.metrics.SparseCategoricalAccuracy()

In [68]:
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = calc_loss(y, logits)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    calc_acc(y, logits)
    return loss

In [69]:
ds = mnist_dataset()
for (x, y) in ds.take(1):
    pass
# train_one_step(model, optimizer, x, y)
model(x)
y

<tf.Tensor: id=4732, shape=(100,), dtype=int32, numpy=
array([9, 5, 0, 7, 5, 3, 3, 7, 2, 9, 0, 8, 3, 6, 6, 2, 1, 1, 8, 5, 7, 1,
       2, 8, 2, 0, 0, 7, 7, 1, 4, 2, 6, 1, 8, 5, 9, 3, 2, 7, 3, 7, 0, 9,
       4, 8, 3, 6, 4, 8, 7, 0, 2, 4, 6, 9, 2, 7, 0, 1, 7, 5, 3, 1, 4, 9,
       1, 2, 1, 1, 0, 7, 0, 6, 6, 5, 9, 9, 9, 9, 8, 4, 5, 1, 5, 2, 9, 7,
       2, 6, 6, 5, 1, 0, 2, 7, 4, 9, 0, 8])>

In [70]:
@tf.function
def train(model, optimizer):
    ds = mnist_dataset()
    step = 0
    loss = 0.0
    accuracy = 0.0
    for x, y in ds:
        step += 1
        loss = train_one_step(model, optimizer, x, y)
        if tf.equal(step % 10, 0):
            acc = calc_acc.result()
            tf.print('Step ', step, 'loss', loss, 'accuracy', acc)
    return step, loss, accuracy

In [71]:
step, loss, accuracy = train(model, optimizer)
print('Final step', step, 'loss', loss, 'accuracy', calc_acc.result())

Step  10 loss 1.82996488 accuracy 0.368
Step  20 loss 1.32712841 accuracy 0.5025
Step  30 loss 0.739079893 accuracy 0.584666669
Step  40 loss 0.864029 accuracy 0.64275
Step  50 loss 0.573883 accuracy 0.6846
Step  60 loss 0.509914279 accuracy 0.712833345
Step  70 loss 0.250975728 accuracy 0.734571457
Step  80 loss 0.327843159 accuracy 0.75425
Step  90 loss 0.46158585 accuracy 0.768666685
Step  100 loss 0.227207169 accuracy 0.7819
Step  110 loss 0.276043683 accuracy 0.792
Step  120 loss 0.455222428 accuracy 0.801
Step  130 loss 0.325037718 accuracy 0.809538484
Step  140 loss 0.370701939 accuracy 0.816357136
Step  150 loss 0.368190408 accuracy 0.8226
Step  160 loss 0.225897133 accuracy 0.828375
Step  170 loss 0.4260104 accuracy 0.833352923
Step  180 loss 0.201803088 accuracy 0.837944448
Step  190 loss 0.222904563 accuracy 0.842631578
Step  200 loss 0.27454102 accuracy 0.8465
Final step tf.Tensor(200, shape=(), dtype=int32) loss tf.Tensor(0.27454102, shape=(), dtype=float32) accuracy tf.Te

In [72]:
def square_if_pos(x):
    return [i ** 2 if i > 0 else i for i in x]

square_if_pos(range(-5, 5))

[-5, -4, -3, -2, -1, 0, 1, 4, 9, 16]

In [74]:
@tf.function
def square_if_pos_vec(x):
    return tf.where(x>0, tf.square(x), x)

square_if_pos_vec(tf.range(-5, 5))

<tf.Tensor: id=5594, shape=(10,), dtype=int32, numpy=array([-5, -4, -3, -2, -1,  0,  1,  4,  9, 16])>